# Groq Text-to-Speech (TTS) Reference

**SAFE REFERENCE NOTEBOOK** - No automatic API calls. Functions and examples only.

This notebook provides clean reference code for Groq PlayAI TTS without making automatic API calls that hit rate limits.

In [5]:
# Environment setup - only loads API key, no calls made
import os
import time
import requests
from IPython.display import Audio, display
from functools import wraps

# Check for API key (no API call made)
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
if not GROQ_API_KEY:
    print("⚠️  Warning: GROQ_API_KEY environment variable not set")
else:
    print("✅ API key found in environment")

AUDIO_DIR = "audio"
os.makedirs(AUDIO_DIR, exist_ok=True)

✅ API key found in environment


## Core TTS Function

Copy this function to use in your projects. Includes retry logic for rate limits.

In [11]:
def retry_on_rate_limit(max_retries=5, base_delay=2.0):
    """Decorator for exponential backoff on rate limits"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            delay = base_delay
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if hasattr(e, 'response') and getattr(e.response, 'status_code', None) == 429:
                        print(f"Rate limit hit. Retrying in {delay:.1f} seconds...")
                        time.sleep(delay)
                        delay *= 2
                    elif '429' in str(e):
                        print(f"Rate limit hit. Retrying in {delay:.1f} seconds...")
                        time.sleep(delay)
                        delay *= 2
                    else:
                        raise
            raise Exception("Max retries exceeded due to rate limits.")
        return wrapper
    return decorator

@retry_on_rate_limit(max_retries=5, base_delay=2.0)
def groq_text_to_speech(text, voice="Celeste-PlayAI", output_path=None):
    """
    Calls the Groq API for text-to-speech and saves the result as a WAV file.
    
    Args:
        text (str): Text to synthesize
        voice (str): PlayAI voice name (see reference below)
        output_path (str): Optional custom output path
    
    Returns:
        str: Path to generated audio file
    """
    api_key = os.environ.get('GROQ_API_KEY')
    if not api_key:
        raise ValueError("Missing GROQ_API_KEY environment variable")
    
    if output_path is None:
        output_path = os.path.join(AUDIO_DIR, "output.wav")
    else:
        output_path = os.path.join(AUDIO_DIR, os.path.basename(output_path))
    
    url = "https://api.groq.com/openai/v1/audio/speech"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "playai-tts",
        "input": text,
        "voice": voice,
        "response_format": "wav"
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 429:
        raise Exception("429 Rate Limit", response)
    if response.status_code != 200:
        raise Exception(f"Groq API error: {response.status_code} {response.text}")

    with open(output_path, "wb") as f:
        f.write(response.content)
    return output_path

# Function is ready to use - NO API CALL MADE YET
print("✅ TTS function loaded and ready")

✅ TTS function loaded and ready


## Example Usage (Run manually when needed)

```python
# Basic usage
audio_path = groq_text_to_speech("Hello world!", voice="Celeste-PlayAI")
display(Audio(audio_path, autoplay=True))

# Custom output path
audio_path = groq_text_to_speech("Hello Ruby!", voice="Ruby-PlayAI", output_path="ruby_test.wav")
display(Audio(audio_path, autoplay=True))
```

## Quick Test Cell

**Run this cell to test the TTS function:**

In [10]:
# Quick TTS test - RUN THIS CELL to actually generate speech
test_text = "Hello! This is a test of Groq TTS."
test_voice = "Celeste-PlayAI"

try:
    print(f"Generating speech: '{test_text}' with voice {test_voice}...")
    audio_path = groq_text_to_speech(test_text, voice=test_voice, output_path="test_speech.wav")
    print(f"✅ Audio generated: {audio_path}")
    display(Audio(audio_path, autoplay=True))
except Exception as e:
    print(f"❌ Error: {e}")

Generating speech: 'Hello! This is a test of Groq TTS.' with voice Celeste-PlayAI...
✅ Audio generated: audio\test_speech.wav
✅ Audio generated: audio\test_speech.wav


## Voice Rolecall Scenario (Run manually)

```python
# Voice rolecall - copy and run when needed
import time

playback = False  # Set to True to hear voices

rolecall_lines = [
    ("Fritz-PlayAI", "Fritz, are you here?", "Here! I'm Fritz, always ready to help with a cheerful attitude."),
    ("Celeste-PlayAI", "Celeste, are you here?", "Here! I'm Celeste, bringing a calm and soothing presence."),
    ("Thunder-PlayAI", "Thunder, are you here?", "Here! I'm Thunder, bold and energetic as always."),
    ("Ruby-PlayAI", "Ruby, are you here?", "Here! I'm Ruby, creative and full of ideas."),
    ("Atlas-PlayAI", "Atlas, are you here?", "Here! I'm Atlas, strong and dependable.")
]

print(f"Starting rolecall scenario with {len(rolecall_lines)} voices...")
for idx, (voice, prompt, reply) in enumerate(rolecall_lines):
    try:
        print(f"[{idx+1}/{len(rolecall_lines)}] {voice} prompt: {prompt}")
        prompt_audio = groq_text_to_speech(prompt, voice=voice, output_path=f"{voice}_prompt.wav")
        if playback:
            display(Audio(prompt_audio, autoplay=True))
        time.sleep(1)
        
        print(f"[{idx+1}/{len(rolecall_lines)}] {voice} reply: {reply}")
        reply_audio = groq_text_to_speech(reply, voice=voice, output_path=f"{voice}_reply.wav")
        if playback:
            display(Audio(reply_audio, autoplay=True))
        time.sleep(1)
        
    except Exception as e:
        print(f"❌ Error with {voice}: {e}")

print("✅ Rolecall scenario complete!")
```

# PlayAI Voices Reference

## All Available Voices (27 total)

**American Accent:**
- `Aaliyah-PlayAI` (female)
- `Angelo-PlayAI` (male)
- `Arista-PlayAI` (female, youthful)
- `Atlas-PlayAI` (male, strong/dependable)
- `Basil-PlayAI` (male)
- `Briggs-PlayAI` (male)
- `Calum-PlayAI` (male)
- `Celeste-PlayAI` (female, calm/soothing)
- `Cheyenne-PlayAI` (female, youthful)
- `Chip-PlayAI` (male)
- `Cillian-PlayAI` (male)
- `Deedee-PlayAI` (female)
- `Eleanor-PlayAI` (female)
- `Fritz-PlayAI` (male, cheerful/helpful)
- `Gail-PlayAI` (female, mature/wise)
- `Jennifer-PlayAI` (female)
- `Judy-PlayAI` (female)
- `Mamaw-PlayAI` (female, elderly/warm)
- `Mason-PlayAI` (male)
- `Mikail-PlayAI` (male)
- `Mitch-PlayAI` (male)
- `Nia-PlayAI` (female)
- `Quinn-PlayAI` (neutral/female)
- `Ruby-PlayAI` (female, creative)
- `Thunder-PlayAI` (male, bold/energetic)

**British Accent:**
- `Adelaide-PlayAI` (female)
- `Indigo-PlayAI` (neutral)

## Recommended Voices for Different Use Cases

**Friendly/Helpful:** Fritz-PlayAI, Ruby-PlayAI
**Professional:** Celeste-PlayAI, Atlas-PlayAI, Gail-PlayAI
**Energetic:** Thunder-PlayAI, Cheyenne-PlayAI
**Mature/Authoritative:** Gail-PlayAI, Mamaw-PlayAI, Atlas-PlayAI
**British Accent:** Adelaide-PlayAI, Indigo-PlayAI

## API Endpoints Reference

**TTS Endpoint:**
```
POST https://api.groq.com/openai/v1/audio/speech
```

**Required Headers:**
```json
{
  "Authorization": "Bearer YOUR_API_KEY",
  "Content-Type": "application/json"
}
```

**Request Body:**
```json
{
  "model": "playai-tts",
  "input": "Text to synthesize",
  "voice": "Celeste-PlayAI",
  "response_format": "wav"
}
```

**Models Endpoint (optional):**
```
GET https://api.groq.com/openai/v1/models
```

## Rate Limiting Tips

- Groq has rate limits on TTS API calls
- Use delays between calls (`time.sleep(1-2)`)
- Implement retry logic with exponential backoff
- Test with small batches first
- Set `playback=False` for faster generation without audio playback

## File Organization

Generated audio files are saved to `audio/` directory by default. Customize with `output_path` parameter.